# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

import pdb

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.user

UserSettings

id: 06643b65-db8b-7d44-8000-96f480b08c58
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'tradier_account_type': None, 'fred_api_key': SecretStr('**********'), 'tradingeconomics_api_key': None, 'nasdaq_api_key': None, 'fmp_api_key': SecretStr('**********'), 'tradier_api_key': None, 'benzinga_api_key': None, 'alpha_vantage_api_key': SecretStr('**********'), 'polygon_api_key': SecretStr('**********'), 'tiingo_token': SecretStr('**********'), 'intrinio_api_key': None, 'biztoc_api_key': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [6]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()

In [8]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [9]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [10]:
symbol = "MRK"
company = "Merck"

In [11]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 06643b65-e02b-752c-8000-a3cc37688356
results: [{'symbol': 'MRK', 'one_day': -0.0038, 'wtd': None, 'one_week': -0.0121, '...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [12]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=-0.0038, wtd=None, one_week=-0.0121, mtd=None, one_month=0.0207, qtd=None, three_month=0.0269, six_month=0.2704, ytd=0.1814, one_year=0.09949999999999999, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.0151, volatility_month=0.0147, price=128.8, volume=3253843.0, average_volume=8210000.000000001, relative_volume=0.47, analyst_recommendation=None, analyst_score=1.48)]

In [13]:
obj.dict()


{'id': '06643b65-e02b-752c-8000-a3cc37688356',
 'results': [{'symbol': 'MRK',
   'one_day': -0.0038,
   'wtd': None,
   'one_week': -0.0121,
   'mtd': None,
   'one_month': 0.0207,
   'qtd': None,
   'three_month': 0.0269,
   'six_month': 0.2704,
   'ytd': 0.1814,
   'one_year': 0.09949999999999999,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.0151,
   'volatility_month': 0.0147,
   'price': 128.8,
   'volume': 3253843.0,
   'average_volume': 8210000.000000001,
   'relative_volume': 0.47,
   'analyst_recommendation': None,
   'analyst_score': 1.48}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 187275875,
   'route': '/equity/price/performance',
   'timestamp': datetime.datetime(2024, 5, 14, 15, 7, 9, 823854)}

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 128.76,
   'bid_size': 800,
   'ask': 128.79,
   'ask_size': 800,
   'last_price': 128.81,
   'open': 129.2,
   'high': 129.19,
   'low': 128.0269,
   'volume': 3364181,
   'prev_close': 129.29,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 126.5478,
   'ma_200d': 114.32535,
   'volume_average': 8238929.0,
   'volume_average_10d': 6842750.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 153446584,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-14T15:07:10.029240'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-15,117.14,117.740,115.49,116.37,5697236.0,116.3517,70438
2023-05-16,115.88,116.760,115.29,116.08,4278281.0,116.2071,60730
2023-05-17,116.37,116.655,113.48,114.76,7210383.0,114.5498,78904
2023-05-18,114.14,115.070,113.33,114.00,6437144.0,114.0864,75952
2023-05-19,114.33,116.240,114.10,115.49,7647080.0,115.4905,63052
...,...,...,...,...,...,...,...
2024-05-07,127.10,130.425,127.07,130.38,6415640.0,129.3458,87045
2024-05-08,130.58,131.510,129.33,129.55,6580073.0,129.7913,74021
2024-05-09,128.94,130.500,128.94,130.23,9038061.0,129.9542,70936


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-15,413.01,413.4300,410.230,412.22,427865,54289383.0,412.1201
2023-05-16,410.25,412.8150,410.240,411.86,447536,57705495.0,411.3774
2023-05-17,415.23,415.8550,410.635,412.35,585038,86786957.0,413.4302
2023-05-18,419.23,419.6700,414.670,414.90,672492,97177195.0,417.1828
2023-05-19,418.62,420.7200,417.350,420.17,642871,103793317.0,419.0232
...,...,...,...,...,...,...,...
2024-05-07,517.14,518.5700,516.450,517.56,432286,50977654.0,517.4968
2024-05-08,517.19,517.7400,515.140,515.26,369150,42012599.0,516.9211
2024-05-09,520.17,520.2074,516.705,517.38,399512,43583253.0,519.1084


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.865,2024-05-14 19:06:50+00:00,56.97,57.3650,56.700,969098,56.76,...,66.1500,54.12,58.6910,60.20040,3086998,1.007705e+11,1772100000,4.61,12.34,2024-05-23 12:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,76.795,2024-05-14 19:07:09+00:00,75.00,77.2700,74.915,5261107,74.70,...,77.2700,48.66,71.8288,63.18105,7037155,1.363925e+11,1776060000,2.39,32.13,2024-07-16 04:00:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,200.780,2024-05-14 19:07:07+00:00,199.00,201.1508,198.160,5502307,198.73,...,201.1508,133.96,192.4348,166.21700,8849290,5.765739e+11,2871670000,16.58,12.11,2024-07-12 12:30:00+00:00
3,MS,Morgan Stanley,NYSE,99.490,2024-05-14 19:07:08+00:00,99.00,100.1799,98.880,4214642,98.56,...,100.1799,69.42,91.4424,85.76010,8010109,1.616872e+11,1625160000,5.50,18.09,2024-07-16 04:00:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,id,amp_url,publisher
date,,,,,,,,,
2024-05-09 13:00:16+00:00,"Investors Heavily Search Merck & Co., Inc. (MR...","Recently, Zacks.com users have been paying clo...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2271265/inves...,MRK,Zacks Equity Research,LThWA5Sh3t7k8BJ8547R71qJY_9Mgkha7WMQAbZTGro,https://www.zacks.com/amp/stock/news/2271265/i...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-10 15:18:00+00:00,Pharma Stock Roundup: PFE DMD Study Patient De...,Pfizer (PFE) reports the death of a participan...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272311/pharm...,"PFE,MRK,LLY",Kinjel Shah,LuH9atBP810MgCmcYvB9qbOEDI5RlsMVZB1l1s1W8D8,https://www.zacks.com/amp/stock/news/2272311/p...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-13 08:06:09+00:00,My Dividend Growth Portfolio: Selling Bio-Phar...,I have shifted my strategy to focus on quality...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4692695-my-di...,"BKNG,TSLA,AMZN,CRM,ZTS,MSCI,ABBV,JNJ,MRK,AVB,O...",Nicholas Ward,KC-Ad56Ahj-rJRZbHcBSSkxF_HxZeV2FXNbZJ0q5R70,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-13 15:16:00+00:00,Bristol Myers (BMY) Fails to Meet Goal in Opdi...,Bristol Myers' (BMY) late-stage label-expandin...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272954/brist...,"AZN,BMY,MRK,LGND",Zacks Equity Research,DZ7EFMYgND4Mcq1fh_0B9LZ6EWP-9-JQz_pR6koaUmA,https://www.zacks.com/amp/stock/news/2272954/b...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-14 13:25:00+00:00,Merck (MRK) Ends Keytruda Combo Melanoma Study...,Merck (MRK) discontinues the Keytruda plus vib...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2273402/merck...,"MRK,MRNA,LGND,ANIP",Zacks Equity Research,R1VrrdEUIYHeJqiOWSzIxRUOlqB_Oa1pGnAYJYBm06w,https://www.zacks.com/amp/stock/news/2273402/m...,{'favicon_url': 'https://s3.polygon.io/public/...


# Question Answering Agent

In [21]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536     
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0



In [22]:
# utility function to call chatgpt
def get_response(messages, tools, model=MODEL, json_format=False):
    """
    Get a single response from ChatGPT based on a chain of messages.

    Args:
        messages (list): A list of message objects representing the conversation history.
        json_format(boolean): True if JSON response requested. (Last message must express the request for JSON response.)

    Returns:
        dict: A response object containing the generated response from ChatGPT.

    Raises:
        OpenAIError: If there is an error during the API call.

    Example:
        >>> messages = [
        ...     {"role": "system", "content": "You are a helpful assistant."},
        ...     {"role": "user", "content": "What's the weather like today?"},
        ... ]
        >>> response = get_response(messages)
    """

    if tools:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",  # auto is default, but we'll be explicit
            response_format={"type": "json_object"} if json_format else None,
        )
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={"type": "json_object"} if json_format else None,
        )

    return response


In [23]:
client = OpenAI()
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

That's a great question! You're referencing a classic line from the film "Monty Python and the Holy Grail." In the movie, the answer isn't straightforward due to the humorous context and the variables involved.

However, if we treat the question scientifically and seriously, we need to specify the type of swallow. Generally, for the common European swallow (*Hirundo rustica*), the estimated airspeed velocity of an unladen swallow is around 20-24 miles per hour (approximately 32-39 kilometers per hour).

It's important to note that this is an approximation and actual speeds can vary based on various factors including wind conditions and the specific behavior of the bird at the time (such as foraging or migration).


In [24]:
# utility functions to support use of client-side tools when querying openai

def eval_tool(tool_call, verbose=True):
    """
    Given an OpenAI tool_call response,
    evaluates the tool function using the arguments provided by OpenAI,
    and returns the message to send back to OpenAI, including the function return value.

    Args:
        tool_call (object): The OpenAI tool_call response.

    Returns:
        dict: The message to send back to OpenAI, containing the tool_call_id, role, name, and value returned by the tool call.

    """
    function_name = tool_call.function.name
    # look up the function based in global tools on the name
    fn = tools[function_name]
    # make the tool call's json args into a dict
    kwargs = json.loads(tool_call.function.arguments)
    
    if verbose:
        print(f"{function_name}({str(kwargs)}) -> ", end="")
    # call function with the args and return value
    fn_value = fn(**kwargs)
    if type(fn_value) is list or type(fn_value) is dict:
        fn_value=str(fn_value)
    if verbose:
        output = str(fn_value)
        if len(output) > 100:
            output=output[:100] + "..."
        print(output)

    return {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": tool_call.function.name,
        "content": fn_value,
    }

def get_response_and_eval(messages, tools=[], json_format=False, raw=False, verbose=False):
    """
    Sends a list of messages to OpenAI and returns the response.
    If tool calls are returned, calls all the tools and sends the values back to OpenAI.
    If further tool calls returned, iterates until no more tool calls are returned and
    'stop' is returned as finish_reason, then returns the response.

    Args:
        messages (list): A list of messages to send to OpenAI.
        json_format (boolean): If the final response should be in JSON format.
        raw (boolean): after last tool is called return raw data response that enabled answering the question
        verbose (bool, optional): If True, prints additional information. Defaults to False.

    Returns:
        response: The final response object returned by OpenAI.

    Raises:
        None

    """
    response = get_response(messages, tools=tools, json_format=json_format)
    choice = response.choices[0]
    response_message = choice.message
    finish_reason = choice.finish_reason

    if verbose:
        print(choice)

    while finish_reason != 'stop':
        # Extend conversation with assistant's reply
        messages.append(response_message)
        if finish_reason == 'tool_calls':
            tool_calls = response_message.tool_calls
            if verbose:
                print(tool_calls)
            # Call the tools and add all return values as messages
            for tool in tool_calls:
                messages.append(eval_tool(tool, verbose=True))
            # Get next response
            response = get_response(messages, tools=tools, json_format=json_format)
            choice = response.choices[0]
            response_message = choice.message
            finish_reason = choice.finish_reason
            if verbose:
                output = str(choice)
                output = output[:1000] + "..." if len(output) > 1000 else output
                print(output)
        else:
            print('finish_reason: ', finish_reason)
            break

    if raw:
        # probably want to process that message and return call signature + value
        return messages[-1]
    else:
        return response

def agent_query(user_message, raw=False, verbose=True):
    """
    Send a user message to OpenAI and retrieve the response, calling all tools until done.

    Args:
        user_message (str): The message from the user.
        raw (boolean): after last tool is called return raw data response that enabled answering the question        
        verbose (bool, optional): Display intermediate tool calls and return values. Defaults to False.

    Returns:
        str: The response from the agent.


    Example:
        >>> agent_query("Hello")
        'Hello! How can I assist you today?'
    """

    # add descriptions of available tools to system prompt
    tool_descs = ""
    openai_tools = []
    for v in tool.agent_registry.values():
        t = v.tooldict
        openai_tools.append(t)
        tname = t['function']['name']
        tdesc = t['function']['description']
        tool_descs += f"{tname} : {tdesc}"
        if v.example_code:
            tool_descs += f" Usage: {v.example_code}"
        tool_descs += "\n---\n\n"
    # tool_descs = "\n".join([f"{tool['function']['name']} : {tool['function']['description']}" for tool in tools.values()])
    current_system_prompt = system_prompt + f"""

Available tools delimited by ---:
{tool_descs}
    """
    # print(current_system_prompt)

    messages = [{"role": "system", "content": current_system_prompt},
                {"role": "user", "content": user_message}]
    response = get_response_and_eval(messages, tools=openai_tools, json_format=False, raw=raw, verbose=verbose)
    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\\$")   # escape stuff that is interpreted as latex
    display(Markdown(response_str))



In [25]:
# for the frequent case where we call a function based on a symbol, make a generic tool class
class BB_agent_tool(object):

    # class variable to keep track of all tools
    agent_registry = {}

    def __init__(self, name, description, openapi_path, parameters, example_parameter_values, callable=None, singular=0):
        self.name = name
        self.description = description
        self.openapi_path = openapi_path
        self.parameters = parameters
        self.example_parameter_values = example_parameter_values
        self.singular = singular
        # needs either openapi_path or callable
        if callable:
            self.callable = callable
        else:
            # get callable via openapi_path
            self.callable = self.get_callable()
        self.tooldict = self.make_tooldict()
        self.example_code = self.make_example_code()
        # store self in agent_registry class variable that keeps track of all instances
        self.agent_registry[self.name] = self

    def __del__(self):
        del self.agent_registry[self.name]

    def __str__(self):
        return (f"Tool Name:          {self.name}\n"
                f"Description:        {self.description}\n"
                f"OpenAPI Path:       {self.openapi_path}\n"
                f"Parameters:         {self.parameters}\n"
                f"Example Parameters: {self.example_parameter_values}\n"
                f"Singular:           {self.singular}\n"
                f"Tooldict:           {self.tooldict}\n"
                f"Examples:           {self.example_code}\n"
                f"Callable:           \n"
                f"{self.callable.__doc__}\n"
        )

    def __call__(self, **kwargs):
        """Make the instance callable and perform the default operation of calling the callable function on a symbol."""
        retval = None
        try:
            # call the tool function
            obj = self.callable(**kwargs)
            # if custom tool, return a list of objects
            if type(obj) is list:
                d = {'results': obj}
            # OpenBB returns objects
            elif obj and obj.results:
                d = json.loads(obj.json())
            # singular should be 0 to return the full array, 1 for first element only, -1 for last element
            if self.singular:
                index = 0 if self.singular == 1 else -1
                retval = json.dumps(d['results'][index])
            else:
                retval = json.dumps(d['results'])
        except Exception as exc:
            print(exc)
        return str(retval)

    def get_callable(self):
        """for /api/vi/equity/search, return the openbb object obb.equity.search"""
        op = obb
        for part in self.openapi_path.removeprefix("/api/v1/").split("/"):
            op = op.__getattribute__(part)
        return op

    def make_tooldict(self):
        """the representation of the tool that is shared with openai"""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": {
                    "type": "object",
                    "properties": self.parameters,
                    "required": [k for k in self.parameters.keys()]   # all required for now
                    }
                }
            }

    def make_example_code(self):
        examples = []
        for example in self.example_parameter_values:
            parray = []
            for k, v in example.items():
                # get type 
                ptype = "string"
                if ptype == "string":
                    parray.append(f'{k}="{v}"') 
                else:
                    parray.append(f'{k}={v}') 
            pstr = ", ".join(parray) 

            # run the example
            return_value = self(**example)
            if type(return_value) is list: # only use 3 values if it's a long list
                return_value = str(return_value[:3])
            if type(return_value) is str:  # truncate long strings
                max_str_len = 1000
                return_value = return_value[:max_str_len] + "..." if len(return_value) > max_str_len else return_value
            
            examples.append(f"{self.name}({pstr}) -> {return_value}")
            return "; ".join(examples)

    def make_metadata(self):
        return {'name': self.name,
                  'description': self.description,
                  'openapi_path': self.openapi_path,
                  'callable': self,
                  'parameters': self.parameters,
                  'example_parameters': self.example_parameter_values,
                  'tooldict': self.tooldict,
                  'example_str': self.example_code}

# Example usage
tool = BB_agent_tool(
    name="get_quote_json",
    description="Given a stock symbol, get the latest market data quote for the stock in JSON format.",
    openapi_path="/api/v1/equity/price/quote",
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

tools = tool.agent_registry

tool(symbol="NVDA")


'[{"symbol": "NVDA", "asset_type": "stock", "name": "NVIDIA CORPORATION COM", "exchange": null, "bid": 914.08, "bid_size": 2, "bid_exchange": null, "ask": 914.25, "ask_size": 2, "ask_exchange": null, "quote_conditions": null, "quote_indicators": null, "sales_conditions": null, "sequence_number": null, "market_center": null, "participant_timestamp": null, "trf_timestamp": null, "sip_timestamp": null, "last_price": 914.14, "last_tick": "down", "last_size": null, "last_timestamp": "2024-05-14T14:50:30", "open": 895.64, "high": 916.51, "low": 889.34, "close": 914.14, "volume": 22993053, "exchange_volume": null, "prev_close": 903.99, "change": 10.15, "change_percent": 0.011228, "year_high": 974.0, "year_low": 281.5199890136719, "iv30": 0.55552, "iv30_change": -0.00861, "iv30_change_percent": -0.015262, "iv30_annual_high": 0.6712400054931641, "hv30_annual_high": 0.7542350006103515, "iv30_annual_low": 0.3138199996948242, "hv30_annual_low": 0.26787900924682617, "iv60_annual_high": 0.5873099899

In [26]:
# make a bespoke tool 

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = json.loads(tool(symbol="MSFT"))
len(tool_response[0]['item1'])

# create tool dict based on metadata
# def make_tool_dict(fn_metadata):
#     retdict  = {
#         "type": "function",
#         "function": {
#             "name": fn_metadata["name"],
#             "description": fn_metadata["description"],
#             "parameters": {
#                 "type": "object",
#                 "properties": fn_metadata["parameters"],
#                 "required": [k for k in fn_metadata["parameters"].keys()]
#                 }
#             }
#         }
#     return retdict

# tools[fn_metadata["name"]] = fn_metadata
# tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


143226

In [27]:
# def create_example_code(fn_metadata):
#     example_str = ""
#     for example in fn_metadata['example_parameters']:
#         if example_str:
#             example_str += " ; "
#         example_str += f"{fn_metadata['name']}("
#         parray = []
#         for k, v in example.items():
#             # get type 
#             ptype = "string"
#             if ptype == "string":
#                 parray.append(f'{k}="{v}"') 
#             else:
#                 parray.append(f'{k}={v}') 
#         example_str += ", ".join(parray) 
#         example_str += ")" 
#         example_str += " -> " 
#         return_value = fn_metadata["callable"](**example)
#         if type(return_value) is list:
#             return_value = str(return_value[:3])
#         if type(return_value) is str:
#             return_value = return_value[:1000] + "..." if len(return_value) > 1000 else return_value
#         example_str += return_value
#         return example_str

# create_example_code(fn_metadata)


In [28]:
# add some stuff about first figure out if it there is a ticker or a company, if not then exit
system_prompt = """
Role: You are an AI stock market assistant tasked with providing up-to-date and detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate and concise answers relevant to their questions about individual stocks.

Capabilities: You are given a number of tools as functions. Use as many tools as needed to ensure all information provided is timely, accurate, concise, relevant, and specific to the user's query.

Instructions: First, analyze the query to fully understand the intent of the user, what data they need, and which tools will satisfy their request.
Then, call the most relevant tools in the order necessary to respond accurately to the investor query. 
If the question is not about a specific stock, if no relevant tool is available, or if uncertain or lacking data, say so clearly and suggest alternative resources or data sources that may help.
Only use data obtained from the tools.

Example Interaction:
Investor: What is the current P/E ratio of Apple Inc.?
AI Response: As of the close of trading on March 10, 2024, Apple Inc.'s P/E ratio is 28.45.
"""


In [29]:
tools

{'get_quote_json': <__main__.BB_agent_tool at 0x30a12cc90>,
 'get_10k_item1_from_symbol': <__main__.BB_agent_tool at 0x30a10e210>}

In [30]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(user_message, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zO42IZLzhrElsvftyo1Rlxwd', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_zO42IZLzhrElsvftyo1Rlxwd', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]
get_10k_item1_from_symbol({'symbol': 'MSFT'}) -> [{"item1": "Note About Forward-Looking Statements\nThis report includes estimates, projections, stat...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="### Summary of Item 1 from Microsoft’s Latest Annual Report\n\n**1. **Business Overview**\n\n**A. Forward-Looking Statements**\n- *Forward-looking statements* throughout the report relating to business plans, objectives, and expected operating results.\n- These s

### Summary of Item 1 from Microsoft’s Latest Annual Report

**1. **Business Overview**

**A. Forward-Looking Statements**
- *Forward-looking statements* throughout the report relating to business plans, objectives, and expected operating results.
- These statements are based on current expectations and subject to risks and uncertainties.

**B. Company Mission**
- Microsoft’s mission: *“to empower every person and every organization on the planet to achieve more.”*
- Focus on creating platforms and tools powered by Artificial Intelligence (AI).

**C. AI and Cloud**
- Expansion with AI capabilities to support business competitiveness, educational and health outcomes, and public sector efficiency.
- Investment spans across Microsoft Teams, Outlook, Bing, Xbox; integration of generative AI for copilot capabilities in services like Microsoft Cloud.
 
**2. What Microsoft Offers**
- Founded in 1975, offers software, services, devices, and solutions.
- Key services include cloud-based solutions, solution support, consulting services, and relevant online advertising.
- Products include operating systems, business applications, software development tools, and video games. Also designs and sells PCs, tablets, gaming consoles, and accessories.

**3. Ambitions and Development Focus**
- *Three inter-connected ambitions*:
    1. Reinvent productivity and business processes.
    2. Build the intelligent cloud and intelligent edge platform.
    3. Create more personal computing.

**4. Commitment to Sustainability**
- Aims to be carbon negative, water positive, and zero waste by 2030.
- Progress updates are detailed in their Environmental Sustainability Report.
 
**5. Commitment to Racial Equity**
- Goals to improve Black and African American community lives, engagement with ecosystem partners, and representation in leadership positions.

**6. Workforce Overview**
- As of June 30, 2023, employs approximately 221,000 people globally.
- Focus on fostering a respectful, rewarding, and inclusive work environment.

**7. Organizational Structure**
- Operates through three segments: *Productivity and Business Processes, Intelligent Cloud, and More Personal Computing*.
- Key products and services, competition, and business strategies are detailed under each segment.

**8. Corporate Social Responsibility Initiatives**
- Efforts include supporting racial equity, digital skills development globally, and robust employee wellbeing programs.

**9. Research and Development**
- Organizes R&D through multiple engineering groups focusing on various aspects like AI, cloud infrastructure, business productivity tools, security, and gaming.
- Strategic importance placed on Microsoft's continued innovation and competitive advantage. 

For more detailed information, Microsoft’s complete annual report provides extensive insight into business strategies, financial results, and market position.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [31]:
# TODO:

# use kwargs argument, use tool class for bespoke funcs , don't use class var sted tools dict, 

# check example_str individually 

# check system prompt

# check additional prompt improvements

# make conversational ... return multiple symbols and have it clarify

# add start_date and end_date for historical queries

# use assistants API


In [32]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [33]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [34]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [35]:
fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            }
        },
    "example_parameter_values": [{
        "query": "Broadcom",
    }],
    "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)


In [36]:
tools['get_equity_search_symbol'](query="Salesforce")

'{"symbol": "CRM", "name": "SALESFORCE INC COM", "dpm_name": "Citadel Securities LLC", "post_station": "5/1"}'

In [37]:
agent_query("What is the stock symbol for Salesforce?", verbose=False)


get_equity_search_symbol({'query': 'Salesforce'}) -> {"symbol": "CRM", "name": "SALESFORCE INC COM", "dpm_name": "Citadel Securities LLC", "post_station"...


The stock symbol for Salesforce is **CRM**.

In [38]:
# agent is able to first get symbol, then get annual report for symbol 
agent_query("What is item 1 from the 10k annual report for Microsoft?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_otpJMjHOFJk6Pey4HfztwHNp', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_otpJMjHOFJk6Pey4HfztwHNp', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]
get_10k_item1_from_symbol({'symbol': 'MSFT'}) -> [{"item1": "Note About Forward-Looking Statements\nThis report includes estimates, projections, stat...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='### Item 1: Business\n\n#### Note About Forward-Looking Statements\nThis report includes estimates, projections, and statements relating to our business plans, objectives, and expected operating results that are “forward-looking statements” within the meaning of 

### Item 1: Business

#### Note About Forward-Looking Statements
This report includes estimates, projections, and statements relating to our business plans, objectives, and expected operating results that are “forward-looking statements” within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. These forward-looking statements generally are identified by words like “believe,” “project,” “expect,” “anticipate,” “estimate,” “intend,” "strategy,” “future,” “opportunity,” “plan,” “may,” “should,” “will,” “would,” “will be,” “will continue,” “will likely result,” and similar expressions. These statements are based on current expectations and assumptions subject to risks and uncertainties that could cause actual results to differ materially.

#### General
Microsoft is a technology company on a mission to empower every person and organization on the planet to achieve more. We strive to create local opportunity, growth, and impact in every country. Microsoft develops and supports software, services, devices, and solutions that deliver new value for customers and help people and businesses realize their full potential. Our offerings include cloud-based solutions that provide customers with software, services, platforms, and content, as well as solution support and consulting services.

#### What We Offer
Founded in 1975, Microsoft offers products including operating systems, productivity and collaboration applications, server applications, business solution applications, desktop and server management tools, and video games. We also design and sell devices, including PCs, tablets, gaming and entertainment consoles, other intelligent devices, and related accessories.

#### The Ambitions That Drive Us
- **Reinvent productivity and business processes:** We help customers create secure, productive work environments, leveraging tools like Office 365, Dynamics 365, and LinkedIn.
- **Build the intelligent cloud and intelligent edge platform:** Azure enables customers to deploy applications globally with higher performance and security.
- **Create more personal computing:** Innovations like Windows 11 and AI-driven applications aim to make computing more personal and engaging.

#### Commitment to Sustainability
Microsoft aims to be a carbon-negative, water-positive, and zero-waste company by 2030. Our sustainability efforts include substantial investments in renewable energy, water replenishment projects, and reducing our carbon footprint.

For further details, you can access the full 10-K report through Microsoft’s Investor Relations website.

In [39]:
agent_query(f"What is the last market quote for symbol {symbol}?", verbose=False)


get_quote_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "stock", "name": "MERCK & CO INC COM", "exchange": null, "bid": 128...


The last market quote for Merck & Co., Inc. (symbol: MRK) is as follows:

- **Last Price**: \\$128.76
- **Bid**: \\$128.75 (size: 3)
- **Ask**: \\$128.77 (size: 3)
- **Open**: \\$129.25
- **High**: \\$129.25
- **Low**: \\$128.03
- **Close**: \\$128.76
- **Previous Close**: \\$129.29
- **Change**: -\\$0.53 (-0.41%)
- **Volume**: 3,244,377

This data is as of May 14, 2024, at 14:51:32.

In [40]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"Can you provide a basic company profile of symbol {symbol}", verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "name": "Merck & Co Inc", "cik": null, "cusip": null, "isin": null, "lei": null, ...


### Company Profile: Merck & Co., Inc. (MRK)

- **Name:** Merck & Co., Inc.
- **Stock Exchange:** NYSE
- **Sector:** Healthcare
- **Industry Category:** Drug Manufacturers - General
- **Segments:**
  - **Pharmaceutical:** Human health pharmaceutical and vaccine products.
  - **Animal Health:** Animal health products including pharmaceutical and vaccine products for disease prevention, treatment, and control in livestock and companion animal species.
  - **Other:** Non-reportable segments of healthcare services.
- **Headquarters:** Rahway, NJ, USA
- **Founded:** 1891
- **Employees:** 72,000
- **Market Cap:** \\$326.23 billion
- **Shares Outstanding:** 2.53 billion
- **Shares Float:** 2.53 billion
- **Short Interest:** 18.75 million shares
- **Institutional Ownership:** 78.09%
- **Beta:** 0.41
- **Indices:** DJIA, S&P 500

### Description:
Merck & Co., Inc. is a healthcare company that provides health solutions through its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. The company operates through the Pharmaceutical, Animal Health, and Other segments.

If you need further details or specific information regarding Merck & Co., Inc., please let me know!

In [41]:
tool = BB_agent_tool(
    name="get_equity_shorts_short_interest",
    description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
    openapi_path='/api/v1/equity/shorts/short_interest',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


get_equity_search_symbol({'query': 'Merck'}) -> {"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station"...
get_equity_shorts_short_interest({'symbol': 'MRK'}) -> [{"settlement_date": "2021-07-15", "symbol": "MRK", "issue_name": "Merck & Co., Inc.", "market_class...


Here are the latest statistics on short interest for Merck & Co., Inc. (symbol: MRK):

- **Settlement Date:** 2024-04-30
- **Current Short Position:** 18,754,018 shares
- **Previous Short Position:** 20,429,972 shares
- **Average Daily Volume:** 7,551,651 shares
- **Days to Cover:** 2.48 days
- **Change in Short Position:** -1,675,954 shares
- **Change Percentage:** -8.20%

These statistics show a recent decrease in the short interest for Merck & Co., Inc.

In [42]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [43]:
agent_query(f"Provide historical split information for symbol AAPL", verbose=False)

get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


The historical stock splits for Apple Inc. (AAPL) are as follows:

1. **August 31, 2020**: 4-for-1 split.
2. **June 9, 2014**: 7-for-1 split.
3. **February 28, 2005**: 2-for-1 split.
4. **June 21, 2000**: 2-for-1 split.
5. **June 16, 1987**: 2-for-1 split.

These splits reflect the adjustments in the number of shares and the share price on the respective dates.

In [44]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what are the latest total assets for symbol {symbol}", verbose=False)


get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


As of the end of the fiscal year on December 31, 2023, Merck & Co., Inc. (symbol: MRK) reported total assets of \\$106,675,000,000 USD.

For more detailed information, you can refer to their [latest 10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [45]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [46]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent cash flow from operations for stock symbol {symbol}", verbose=False)


get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent cash flow from operations for Merck & Co., Inc. (MRK) was \\$13,006,000,000 for the fiscal year ending December 31, 2023.

For further details, you can review the [full filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [47]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent net income for symbol {symbol}", verbose=False)


get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


As of the fiscal year ending December 31, 2023, Merck & Co., Inc. (symbol: MRK) reported a net income of \\$365 million. For further details, you can refer to their latest [10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm) on the SEC website.

In [48]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the most recent PE ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 326250000000.0, "pe_ratio": 143.39, "foward_pe": 13.0, "eps": 0.9, ...


As of the most recent data, the P/E ratio for Merck & Co., Inc. (symbol: MRK) is 143.39.

In [49]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent price to sales ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 326250000000.0, "pe_ratio": 143.39, "foward_pe": 13.0, "eps": 0.9, ...


As of the most recent data, the price-to-sales ratio for Merck & Co., Inc. (MRK) is 5.34.

In [50]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent revenue per share for stock symbol {symbol}", verbose=False)


get_equity_search_symbol({'query': 'Merck'}) -> {"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station"...
get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol": "MRK", "revenue_per_share_ttm": 24.25424397947098, "net_income_per_share_ttm": 0.91038294...


As of the most recent data, the revenue per share for Merck & Co., Inc. (symbol: MRK) is \\$24.25.

In [51]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# 

In [52]:
# agent_query(f"what was the EPS for {company} in the quarter ended 2022-09-30?", verbose=False)


In [53]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the latest dividend data for stock symbol {symbol}?", verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for stock symbol MRK (Merck & Co., Inc.) is as follows:

- **Ex-Dividend Date:** March 14, 2024
- **Dividend Amount:** \\$0.77
- **Record Date:** March 15, 2024
- **Payment Date:** April 5, 2024
- **Declaration Date:** January 23, 2024

If you have further questions or need additional information, feel free to ask!

In [54]:
agent_query(f"what was the dividend data as of the end of 2022 for stock symbol {symbol}?", verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


As of the end of 2022, the dividend data for Merck & Co., Inc. (MRK) is as follows:

- **Ex-Dividend Date**: December 14, 2022
- **Dividend Amount**: \\$0.73
- **Record Date**: December 15, 2022
- **Payment Date**: January 9, 2023
- **Declaration Date**: November 29, 2022

In [55]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", verbose=False)


get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-12", "trailing_dividend_yield": 0.0242444938}, {"date": "2023-05-15", "trailing_d...


As of the end of 2023, Merck & Co., Inc. (MRK) had a trailing dividend yield of approximately 2.82%.

In [56]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the performance for {company} from 1 year ago?", verbose=False)


get_equity_search_symbol({'query': 'Merck'}) -> {"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station"...
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": -0.0037, "wtd": null, "one_week": -0.0121, "mtd": null, "one_month": 0...


Over the past year, Merck & Co Inc. (MRK) has experienced a performance increase of approximately 9.96%. The current price stands at \\$128.81, with a volume of 3,264,940 shares compared to an average volume of 8,210,000 shares.

In [57]:
# this might exceed token context making it unreliable
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [58]:
pd.DataFrame(json.loads(tool(symbol='MRK'))).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
436,MRK,FTXH,11684.0,0.075700,1.519621e+06
76,MRK,QDVG.DE,1117567.0,0.064076,1.466248e+08
77,MRK,IUHE.AS,0.0,0.062564,1.460573e+08
78,MRK,IUHC.L,1121581.0,0.062374,1.453008e+08
13,MRK,XLV,18454434.0,0.062123,2.400184e+09
...,...,...,...,...,...
560,MRK,GXUS,2163.0,NaN,3.629300e+05
661,MRK,GPAL,640.0,NaN,6.631040e+04
672,MRK,GDEF,397.0,NaN,5.170131e+04
679,MRK,EUNU.DE,50000.0,NaN,3.868500e+04


In [59]:
agent_query(f"which ETF has the highest weight in {company} ?", verbose=False)


get_equity_search_symbol({'query': 'Merck'}) -> {"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station"...
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79663326.0, "weight": 0.0066, "market_val...


The ETF with the highest weight in Merck (MRK) is the **Health Care Select Sector SPDR Fund (XLV)**, which has a weight of approximately 6.21% in Merck.

In [66]:
[(k, t.description) for k, t in tools.items()]

[('get_quote_json',
  'Given a stock symbol, get the latest market data quote for the stock in JSON format.'),
 ('get_10k_item1_from_symbol',
  "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing."),
 ('get_equity_search_symbol',
  'Given a search string, get the stock symbol of the top company whose name best matches the search string.'),
 ('get_company_profile_json',
  'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'),
 ('get_equity_shorts_short_interest',
  'Given a stock symbol, get data on short volume and days to cover in JSON format.'),
 ('get_equity_fundamental_historical_splits',
  "Given a stock symbol, get the company's historical stock splits in JSON format."),
 ('get_balance_sheet_json',
  'Given a stock symbol, get the latest balance sheet data for the company in JSON format.'),
 ('get_cash_flow_json',
  'Given a stock symbol, get the latest cash flow state